# Selenium Challenge
<p>Oh no! You need to scrape and work with some data on a website that you used to frequent. The developers stuck it behind a login screen! How are you going to work with that data now??? Time to throw away beautiful soup and break out selenium.</p>
<p>Use selenium to bypass the login screen so that you can automate the process of answering the following 3 questions.</p>
<p>Challenge URL: <a href="https://modest-jennings-77f32e.netlify.app/">https://modest-jennings-77f32e.netlify.app/</a></p>
<p>Your password: jZ3HF+}g</p>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep

import requests

import unicodecsv as csv

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [58]:

class Player:
    def __init__(self, name, team, position, age, games_played, minutes_per_game):
        self.name = name
        self.team = team
        self.position = position
        self.age = age
        self.games_played = games_played
        self.minutes_per_game = minutes_per_game
        self.oldest_player = None
        
    @classmethod
    def from_dict(cls, my_dict):
        return cls(my_dict['name'], my_dict['team'],
                   my_dict['position'], my_dict['age'],
                   my_dict['games_played'], my_dict['minutes_per_game'])
    
    def __repr__(self):
        return f'<{self.name} {self.team} {self.age} {self.minutes_per_game}>'
    
class Team: 
    def __init__(self, name, players=None):
        self.name = name
        
        if players is None:
                self.players = []
        else:
            self.players = players
        
    def add_player(self, player):
        self.players.append(player)
    
    def remove_player(self, player_name):
        for idx, player in enumerate(self.players):
            if player.name == player_name:
                self.players.pop(idx)
                
    def average_age(self):
        total_age = 0
        for player in self.players:
            total_age += float(player.age)
            
        return total_age / len(self.players)
    
    def oldest_player(self):
        playerAges = {player.name: player.age for player in self.players}
        return max(playerAges, key=playerAges.get)

    def least_played(self):
        leastPlayed = {player.name: player.games_played for player in self.players if 'F' in player.position}
        return min(leastPlayed, key=leastPlayed.get)
                
    def __repr__(self):
        return f'<Team: {self.name.upper()} Players: {len(self.players)}>'

Begin scraping...
[Evan Bos 28.72 33.3, Aaron Bos 21.76 15, Payton Bos 23.48 13.5, Marcus Bos 27.38 36.6, Jayson Bos 23.38 37, Tristan Bos 30.36 26.4, Grant Bos 22.64 11.4, Romeo Bos 21.74 27.2, Jabari Bos 26.35 14.8, Kemba Bos 31.2 30.3, Tremont Bos 23.53 2, Robert Bos 23.76 15.2, Carsen Bos 23.36 2.7, Luke Bos 26.02 2.7, Semi Bos 26.63 6, Tacko Bos 25.61 0.9, Bogdan Atl 28.92 33.2, Clint Atl 27.18 31.6, John Atl 23.83 32, Danilo Atl 32.95 24.6, Kevin Atl 22.9 31, Onyeka Atl 20.61 9.2, Lou Atl 34.73 15.4, Trae Atl 22.84 37.7, Solomon Atl 30.34 10.4, Tony Atl 29.69 7.3, Skylar Atl 23.87 2.4, Bruno Atl 22.93 2, Nathan Atl 23.83 2.3, De'Andre Atl 23.63 30.3, Kris Atl 27.34 6.6, Cam Atl 21.89 23, Jalen Dal 24.89 16.3, Luka Dal 22.39 40.2, Dorian Dal 28.21 38.7, Tim Dal 29.35 37.4, Maxi Dal 29.47 26.8, Kristaps Dal 25.97 33.3, Dwight Dal 30 7.5, Josh Dal 27.85 13.4, Willie Dal 27.92 10.5, Boban Dal 32.93 20.9, Nicolo Dal 30.48 6.4, Trey Dal 28.69 8.3, Josh Dal 20.67 4.2, Facundo Den 30.33 

In [3]:
# # open url and enter data by entering password and click login
# driver = webdriver.Chrome('./chromedriver')
# driver.get('https://modest-jennings-77f32e.netlify.app/data')
# sleep(3)
# driver.find_element_by_css_selector('#password').send_keys('jZ3HF+}g')
# driver.find_element_by_css_selector("button").click()
# sleep(3)
# player_row = driver.find_element_by_css_selector('table.min-w-full tbody tr')

In [46]:
def get_player_data():
    print("Begin Scraping...")
    driver = webdriver.Chrome('./chromedriver')
    driver.get('https://modest-jennings-77f32e.netlify.app/data')
    sleep(1)
    driver.find_element_by_css_selector('#password').send_keys('jZ3HF+}g')
    driver.find_element_by_css_selector("button").click()
    sleep(1)
    
    rows = driver.find_elements_by_css_selector(".min-w-full tbody > tr")
    
    for player in rows:
        p = player.text.split()
        
        player_dict = {
            "name": p[1],
            "team": p[4] if p[3] in ['Jr.','III', 'II', 'Sr.'] else p[3],
            "position": p[5] if p[3] in ['Jr.','III', 'II', 'Sr.'] else p[4],
            "age": p[6] if p[3] in ['Jr.','III', 'II', 'Sr.'] else p[5],
            "games_played": p[7] if p[3] in ['Jr.','III', 'II', 'Sr.'] else p[6],
            "minutes_per_game": p[8] if p[3] in ['Jr.','III', 'II', 'Sr.'] else p[7],
        }
        
#         player_dict = {
#         "name": driver.find_element_by_css_selector('table.min-w-full tbody.bg-white tr[2] ').text,
#         "team": driver.find_element_by_css_selector('table.min-w-full tbody.bg-white tr[3] ').text,
#         "position": driver.find_element_by_css_selector('table.min-w-full tbody.bg-white tr[4] ').text,
#         "age": driver.find_element_by_css_selector('table.min-w-full tbody.bg-white tr[5] ').text,
#         "games_played": driver.find_element_by_css_selector('table.min-w-full tbody.bg-white tr[6] ').text,
#         "minutes_per_game": driver.find_element_by_css_selector('table.min-w-full tbody.bg-white tr[7] ').text
#         }
        
        new_player = Player.from_dict(player_dict)
        
            
        players.append(new_player)
        
    driver.close()
        
    print(players)
    
    print('End Scraping')
   
    return players
    
get_player_data('')

TypeError: get_player_data() takes 0 positional arguments but 1 was given

In [10]:
def get_most_mpg(players):
    
    most_mpg = 0
    
    for player in players:
        mpg = player.get_mpg()

        if float(mpg) > float(most_mpg):
            most_mpg = mpg
    for player in players:
      
        if player.minutes_per_game == most_mpg:
            return player
        else:
            continue
        
    return 'Error'

In [48]:
def get_most_mpg(players):
    
    most_mpg = 0
    
    for player in players:
        mpg = player.get_mpg()

        if float(mpg) > float(most_mpg):
            most_mpg = mpg
    for player in players:
      
        if player.minutes_per_game == most_mpg:
            return player
        else:
            continue
        
    return 'Error'

In [ ]:
oldest_of_teams = {}

for teamName in teams:
    oldest_of_teams[teamName] = teams[teamName].oldest_player()

#     if teamName not in oldest_of_teams:
#         oldest_of_teams[teamName] = teams[teamName]._oldest_player()

In [13]:
least_played_f = {}

for teamName in teams:
    least_played_f[teamName] = teams[teamName].least_played()

## Question 1
<p>Find the player with the most MPG in the league</p>

In [42]:
print(get_most_mpg(players))

Damian Por 31.02 41.3


## Question 2
<p>Please find the name of the oldest player on each team. Add them to a dictionary with each team's name being a key and the players name being the value.</p>

In [ ]:
# Ex:
oldest_players = {
    "Atl": "Lou Williams",
    "Bos": "Tristan Thompson",
    # Etc for all teams
}

In [40]:
print(oldest_of_teams)

{'Bos': 'Kemba', 'Atl': 'Lou', 'Dal': 'Boban', 'Den': 'Paul', 'Lac': 'Rajon', 'Lal': 'LeBron', 'Mem': 'Jonas', 'Mia': 'Andre', 'Mil': 'P.J.', 'Nyk': 'Taj', 'Phi': 'Anthony', 'Pho': 'Chris', 'Por': 'Carmelo', 'Uta': 'Ersan', 'Was': 'Robin', 'Bro': 'Jeff'}


## Question 3
<p>Find the player with position F that had the least Games Played (GP) for each team, save as a dictionary. If there are multiple with the same GP, pick one.</p>

In [ ]:
# Ex:
most_played_center = {
    "Atl": "Solomon Hill",
    "Bos": "Semi Ojeleye",
    "Bro": "Reggie Perry",
    "Dal": "Josh Green", 
    "Den": "Bol Bol",
    "Lac": "Serge Ibaka", 
    "Lal": "Jared Dudley",
    "Mem": "Justice WInslow", 
    "Mia": "Max Strus",
    "Mil": "Donte DiVincenzo",
    "Nyk": "Kevin Knox II",
    "Phi": "Anthony Tolliver",
    "Pho": "Ty-Shon Alexander",
    "Por": "Harry Giles III", 
    "Uta": "Ersan Ilyasova",
    "was": "Cassius Winston"
    

    # Etc for all teams
}

In [44]:
print("least_played_f")

{'Bos': 'Luke', 'Atl': 'Solomon', 'Dal': 'Nicolo', 'Den': 'Aaron', 'Lac': 'Amir', 'Lal': 'Jared', 'Mem': 'Justise', 'Mia': 'Nemanja', 'Mil': 'Thanasis', 'Nyk': 'Kevin', 'Phi': 'Anthony', 'Pho': 'Frank', 'Por': 'Harry', 'Uta': 'Udoka', 'Was': 'Chandler', 'Bro': 'Bruce'}
